First we import `pandas` for dealing with data and `os` to deal with the operating system. We then set the working directory for the notebook. `os` comes out the box with python but you'll need to install `pandas` if you don't have it already. You should change `WDIR` to the working directory of wherever you have cloned this repository. 

In [8]:
import pandas as pd
import os
# setting up the working directory
WDIR = 'c:\\Users\\LukeStrange\\Code\\solid-succotash'
os.chdir(WDIR)

## setting the year for the data we want to use
year = '2022'

Below is a function called `transform`. This reads the data and transforms it into the required shape for further processing. 

In [9]:
def transform(path, sheet_name):
    '''
    Read the data from excel and transform it.
    Must have columns specified in "cols" and these must be on row 1 of the sheet.
    
    Params
    ------
        path: the relative file-path to the data file
        sheet_name: the name of the sheet to read
    '''
    cols=['Substation', 'Supply Level', 'Scenario', year]
    data = pd.read_excel(path, sheet_name=sheet_name)
    # make sure all column headings are strings
    data.columns = data.columns.astype(str)

    # for col in cols:
    #     assert data.columns.to_list(), f'Data file does not contain the column {col}'
    try:
        data = data.filter(regex='([S-s]cenario|[S-s]upply [L-l]evel|[S-s]ubstation|2022)', axis='columns')
    except:
        print('Please check your column names. Unable to find one or more of the following column headings', cols)
        return 
    # filtering by "best view" scenario and Primary substations only
    data = data[(data['Scenario'].str.contains('best view', case=False)) & (data['Supply Level'].str.contains('primary', case=False))].reset_index(drop=True)
    #data.drop(columns='index', inplace=True)
    return data

# calling the function on each dataset, and setting it as a variable.
east_mids = transform(r'data-raw/headroom-capacity/East Midlands Network Headroom Report.xlsx', sheet_name='Demand headroom')
south_wales = transform(r'data-raw/headroom-capacity/South Wales Network Headroom Report.xlsx', sheet_name='Demand headroom')
south_west = transform(r'data-raw/headroom-capacity/South West Network Headroom Report.xlsx', sheet_name='Demand headroom')
west_mids = transform(r'data-raw/headroom-capacity/West Midlands Network Headroom Report.xlsx', sheet_name='Demand headroom')
north = transform(r'data-raw/headroom-capacity/north_yorkshire_southern_northscotland_scottish.xlsx', sheet_name='NPgN Demand headroom')
yorkshire = transform(r'data-raw/headroom-capacity/north_yorkshire_southern_northscotland_scottish.xlsx', sheet_name='NPgY Demand headroom')
southern = transform(r'data-raw/headroom-capacity/north_yorkshire_southern_northscotland_scottish.xlsx', sheet_name='SEPD Demand headroom')
north_scotland = transform(r'data-raw/headroom-capacity/north_yorkshire_southern_northscotland_scottish.xlsx', sheet_name='SHEPD Demand headroom')
southern_scotland = transform(r'data-raw/headroom-capacity/north_yorkshire_southern_northscotland_scottish.xlsx', sheet_name='SPD Demand headroom')
north_west = transform(r'data-raw/headroom-capacity/northwestern.xlsx', sheet_name='Demand Primary Headroom')
north_wales_merseyside = transform(r'data-raw/headroom-capacity/merseyside_northern_wales.xlsx', sheet_name='Baseline Scenario (Demand)')
east = transform(r'data-raw/headroom-capacity/eastern_london_southeastern.xlsx', sheet_name='Eastern demand')
south_east = transform(r'data-raw/headroom-capacity/eastern_london_southeastern.xlsx', sheet_name='South Eastern demand')
london = transform(r'data-raw/headroom-capacity/eastern_london_southeastern.xlsx', sheet_name='London demand')
# show an example of the transformed frame
east_mids
# @TODO add scotland


,Substation,Supply Level,Scenario,2022
0,COURTAULDS 33 11kV S STN,Primary,WPD Best View,0.65
1,COX STREET 33 6 6kV S STN,Primary,WPD Best View,10.41
2,Courthouse Green 33/11kv S Stn,Primary,WPD Best View,8.27
3,Courthouse Green 33/6 6kv S Stn,Primary,WPD Best View,4.17
4,DUNLOP 33 6 6kV S STN,Primary,WPD Best View,14.33
...,...,...,...,...
377,CROMFORD 33 11kV S STN,Primary,WPD Best View,8.63
378,HOPTON 33 11kV S STN,Primary,WPD Best View,-1.54
379,LONGCLIFFE 33 11kV S STN,Primary,WPD Best View,12.12
380,MATLOCK 33 11kV S STN,Primary,WPD Best View,11.59


Below defines a function `network_scenario_bins` which puts each value from the 2022 capacity column into a bin.

In [10]:
def network_scenario_bins(data, place):
    '''
        Take the transformed dataframe and a place name 
        and return the binned network capacity values.
        
        Params
        ------
        data: dataframe
        place: str
        
        Returns
        -------
        conc: dataframe
    '''
    # set up the bins and labels
    bin_labels = ['-60 to -50', '-50 to -40', '-40 to -30', '-30 to -20','-20 to -10', '-10 to 0', '0 to 10', '10 to 20', '20 to 30', '30 to 40', '40 to 50', '50 to 60', '60 to 70', '70 to 80']
    bins = [-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80]

    # bin the data and add it as a new column
    data['network_capacity'] = pd.cut(data[year], bins=bins, precision=4, labels=bin_labels).reset_index()[year]

    # count the the number of values in each bin
    count_in_bin = data['network_capacity'].value_counts().reset_index()

    # set the place name as a column
    count_in_bin['place'] = f'{place}'

    # return the pivoted wide-format dataframe
    return count_in_bin.pivot(index='place', columns='network_capacity', values='count')

Next we call the function `network_scenario_bins` on each 

In [11]:
east_mids_bins = network_scenario_bins(east_mids, 'East Midlands')
south_wales_bins = network_scenario_bins(south_wales, 'South Wales')
south_west_bins = network_scenario_bins(south_west, 'South West')
west_mids_bins = network_scenario_bins(west_mids, 'West Midlands')
north_bins = network_scenario_bins(north, 'North')
yorkshire_bins = network_scenario_bins(yorkshire, 'Yorkshire')
southern_bins = network_scenario_bins(southern, 'Southern')
north_scotland_bins = network_scenario_bins(north_scotland, 'Northern Scotland')
southern_scotland_bins = network_scenario_bins(southern_scotland, 'Southern Scotland')
north_west_bins = network_scenario_bins(north_west, 'North West')
north_wales_merseyside_bins = network_scenario_bins(north_wales_merseyside, 'North Wales & Merseyside')
east_bins = network_scenario_bins(east, 'East')
london_bins = network_scenario_bins(london, 'London')
south_east_bins = network_scenario_bins(south_east, 'South East')

In [12]:
pd.concat([east_mids_bins, 
           south_wales_bins, 
           south_west_bins, 
           west_mids_bins, 
           north_bins, 
           yorkshire_bins, 
           southern_bins, 
           south_east_bins, 
           southern_scotland_bins, 
           north_scotland_bins, 
           north_west_bins, 
           north_wales_merseyside_bins, 
           east_bins, 
           london_bins])

network_capacity,-60 to -50,-50 to -40,-40 to -30,-30 to -20,-20 to -10,-10 to 0,0 to 10,10 to 20,20 to 30,30 to 40,40 to 50,50 to 60,60 to 70,70 to 80
place,,,,,,,,,,,,,,
East Midlands,1,0,0,2,4,22,225,118,8,0,0,1,0,1
South Wales,0,0,0,1,0,5,119,48,9,1,0,0,0,0
South West,0,0,0,0,1,13,212,62,9,4,0,0,0,0
West Midlands,1,0,0,0,1,16,85,34,23,19,13,6,0,0
North,0,0,0,0,0,0,79,77,18,5,0,0,0,0
Yorkshire,0,0,0,0,0,3,167,177,18,1,0,0,0,0
Southern,0,0,0,0,0,35,244,110,16,4,1,2,0,0
South East,0,0,0,0,0,10,166,50,14,0,2,0,0,0
Southern Scotland,0,0,0,0,0,4,191,184,3,1,0,0,0,0


Next we define a function that calculates the number of sites in each region that had either positive capacity (headroom) or negative capacity (overload) and the total amount of capacity (positive = headroom, negative = overload) for each region.

In [13]:
def scenario_sum_count(data, place):
    '''
        Take the transformed dataframe and a place name 
        and return the network capacity sum total and 
        count total.
        
        Params
        ------
        data: dataframe
        place: str
        
        Returns
        -------
        conc: dataframe
    '''
    overload_sum = data[data[year] < 0].sum(numeric_only=True)
    headroom_sum = data[data[year] >= 0].sum(numeric_only=True)
    total_capacity = pd.DataFrame(data={'overload_sum': overload_sum, 'headroom_sum': headroom_sum})
    total_capacity.rename(index={year: f'{place}_{year}'}, inplace=True)
    #print(total_capacity)
    #total_capacity.drop('index', inplace=True)

    headroom_count = data[data[year] > 0].count(axis=0)
    overload_count = data[data[year] < 0].count(axis=0)
    df = pd.DataFrame(data={'overload_count': overload_count, 'headroom_count': headroom_count})
    #print(df.columns)
    #print(df.index.to_list())
    df.drop(index=['Substation', 'Supply Level', 'Scenario', 'network_capacity'], inplace=True)
    df.rename(index={year: f'{place}_{year}'}, inplace=True)
    conc = pd.concat([total_capacity, df], axis=1)
    conc.index.rename('dno_license_area', inplace=True)
    return conc.round(1)

Finally we call that function for each DNO license area.

In [14]:
east_mids_sum_count = scenario_sum_count(east_mids, 'East Midlands')
west_mids_sum_count = scenario_sum_count(west_mids, 'West Midlands')
south_wales_sum_count = scenario_sum_count(south_wales, 'South Wales')
south_west_sum_count = scenario_sum_count(south_west, 'South West')
north_sum_count = scenario_sum_count(north, 'North')
yorkshire_sum_count = scenario_sum_count(yorkshire, 'Yorkshire')
south_east_sum_count = scenario_sum_count(south_east, 'South East') 
southern_sum_count = scenario_sum_count(southern, 'South')
southern_scotland_sum_count = scenario_sum_count(southern_scotland, 'Southern Scotland')
north_scotland_sum_count = scenario_sum_count(north_scotland, 'Northern Scotland')
north_west_sum_count = scenario_sum_count(north_west, 'North West')
north_wales_merseyside_sum_count = scenario_sum_count(north_wales_merseyside, 'North Wales and Merseyside')
east_sum_count = scenario_sum_count(east, 'East')
london_sum_count = scenario_sum_count(london, 'London')

# concatenate all the dataframes together
merged_df = pd.concat([north_sum_count, 
                       yorkshire_sum_count, 
                       east_mids_sum_count, 
                       west_mids_sum_count, 
                       south_wales_sum_count, 
                       south_east_sum_count, 
                       south_west_sum_count,
                       southern_sum_count,
                       southern_scotland_sum_count,
                       north_scotland_sum_count,
                       north_west_sum_count,
                       north_wales_merseyside_sum_count,
                       east_sum_count,
                       london_sum_count])
# write the data to a csv file.
merged_df.to_csv('src/questions/4/_data/headroom.csv')
merged_df

,overload_sum,headroom_sum,overload_count,headroom_count
dno_license_area,,,,
North_2022,0.0,2201.5,0,179
Yorkshire_2022,-1.2,3865.8,2,363
East Midlands_2022,-211.8,3202.6,28,353
West Midlands_2022,-114.8,3450.0,18,185
South Wales_2022,-39.7,1521.3,6,177
South East_2022,-8.1,1896.3,10,232
South West_2022,-44.4,2105.6,14,287
South_2022,-51.6,3290.8,35,377
Southern Scotland_2022,-0.8,3651.1,4,379


# Additional Notes and recommendations
Firstly, this script is not generalised. It requires the initial data to be in a specific format with specific columns. I have tried to match these case-insensitively, but ideally the source data would use the exact column headings.

You could feasibly automate scraping all the differnet DNO websites and make one big data table in long format. This would need an additional column called something like DNO_LA (DNO license area). This would ensure that all the data for each DNO was in the same format with the same column headings. For example, the data from national grid is all in the same format and so could be stored in one long spreadsheet rather than several sheets. This would make the data _much_ more machine-readable, and probably reduce the run time of this script by a factor of ~100.

The outputs of this script are stored in a csv file - again this is a very machine-friendly file format but can also be opened in excel. It drives the visualisations on the page of Q4. If the data were to change, the visualisations would change - so in a sense they are "live". If we were to build a full pipeline for the data then you could have new analysis and charts in the time it takes to run the scripts (currently a few seconds). 

## General data tips
- Be consistent. The number one problem with automation is when the expected data format changes. A machine doesn't know the differnce between  a capital letter and a lower case one unless you explicitly tell it what to look for. Something as simple as "Best View" vs "best view" could cause a programme to fail. Indeed, we try to make code more robust than that, but its impossible to account for every unknown.
- Write column titles in snake case. For example "Supply Level" would become "supply_level". Or DNO License Area would be "dno_license_area"
- Try and ensure the header row is row 1 of the table.
- Limit formatting as much as possible - it just makes it harder to read using code.